# Рекомендация тарифов

### Описание проекта:
Оператор мобильной связи выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям один из новых тарифов. В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.

### Цель проекта:
Построить модель для задачи классификации с максимально большим значением accuracy, которая выберет подходящий тариф. Нужно довести долю правильных ответов по крайней мере до 0.75.

## Загрузка данных и изучение общей информации

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/data/users_behavior.csv')

***Описание данных:***

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
data.query('calls == 0 and minutes == 0').head(20)

,calls,minutes,messages,mb_used,is_ultra
54,0.0,0.0,33.0,14010.33,1
247,0.0,0.0,35.0,16444.99,1
264,0.0,0.0,21.0,19559.55,0
351,0.0,0.0,8.0,35525.61,1
390,0.0,0.0,25.0,19088.67,1
484,0.0,0.0,191.0,32448.02,1
551,0.0,0.0,24.0,18701.54,1
647,0.0,0.0,30.0,10236.20,1
769,0.0,0.0,32.0,17206.44,0
884,0.0,0.0,180.0,32045.73,1


In [7]:
data.duplicated().sum()

0

#### Вывод:
1. Пропуски, нарушения хорошего стиля, явные дубликаты не обнаружены. 
2. Присутствуют нулевые значения в колонках звонки и минуты - возможно это бизнес-аккаунты  с использованием мессенджеров или просто пользователи исключительно интернет-трафика, но по заданию преобработка данных не требуется, будем считать, что на адекватность модели это не повлияет.
3. Всего 3214 строки, в каждой содержится информация по каждому пользователю об используемых ресурсах за месяц.
Данные готовы к работе.

## Разделение данных на выборки

Разобъем данные на тренировочную (60%), валидационную (20%) и тестовую (20%) выборки:

In [8]:
data_train, data_valid_test = train_test_split(
    data, test_size=0.40, random_state=12345)

data_test, data_valid = train_test_split(
    data_valid_test, test_size=0.50, random_state=12345)

print(data_train.shape)
print(data_test.shape)
print(data_valid.shape)

(1928, 5)
(643, 5)
(643, 5)


Определим целевой признак - тариф, колонка 'is_ultra'. 

Т к значений целевого признака всего два - определим тип задачи - **бинарная классификация**.

Выборки готовы, извлечем признаки и целевой признак для каждой выборки (train - выборка для обучения модели, valid - валидационная выборка и test - тестовая выборка):

In [9]:
train_features = data_train.drop(['is_ultra'], axis=1) 
train_target = data_train['is_ultra']

valid_features = data_valid.drop(['is_ultra'], axis=1) 
valid_target = data_valid['is_ultra']

test_features = data_test.drop(['is_ultra'], axis=1) 
test_target = data_test['is_ultra']

## Исследование моделей

Для решения задачи обучим три типа моделей и проверим качество прогнозирования:
* Дерево решений
* Случайный лес
* Логистическая регрессия

Затем лучшую модель с  долей правильных ответов не менее 0.75 проверим на тестовой выборке.

### Дерево решений

In [10]:
best_model = None
best_result = 0
for depth in range(2, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(train_features, train_target) 
    result = model.score(valid_features, valid_target)
    if result > best_result:
            best_model = model
            best_result = result
            best_depth = depth
          

print('Качество наилучшей модели на валидационной выборке:', best_result, '\nГлубина дерева:', best_depth)

Качество наилучшей модели на валидационной выборке: 0.7993779160186625 
Глубина дерева: 7


In [11]:
best_model = None
best_result = 0
for depth in range(2, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=3) # обучите модель с заданной глубиной дерева
    model.fit(train_features, train_target) 
    result = model.score(valid_features, valid_target)
    if result > best_result:
            best_model = model
            best_result = result
            best_depth = depth
          

print('Качество наилучшей модели на валидационной выборке:', best_result, '\nГлубина дерева:', best_depth)

Качество наилучшей модели на валидационной выборке: 0.8009331259720062 
Глубина дерева: 7


### Случайный лес

In [12]:
best_model = None
best_result = 0

for depth in range (1, 10):
    for est in range(1, 30):
        model = RandomForestClassifier(random_state=12345, criterion='gini', n_estimators=est, max_depth=depth) 
        model.fit(train_features, train_target) 
        result = model.score(valid_features, valid_target)
        if result > best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_est = est

print('Качество наилучшей модели на валидационной выборке:', best_result, '\nГлубина дерева:', best_depth, 
      '\nКоличество деревьев', best_est)

Качество наилучшей модели на валидационной выборке: 0.8149300155520995 
Глубина дерева: 9 
Количество деревьев 14


In [13]:
best_model = None
best_result = 0

for depth in range (1, 10):
    for est in range(1, 30):
        model = RandomForestClassifier(random_state=12345, criterion='gini', n_estimators=est, max_depth=depth) 
        model.fit(train_features, train_target)
        train_predictions = model.predict(valid_features)
        result = accuracy_score(valid_target, train_predictions)
        if result > best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_est = est

print('Качество наилучшей модели на валидационной выборке:', best_result, '\nГлубина дерева:', best_depth, 
      '\nКоличество деревьев', best_est)

Качество наилучшей модели на валидационной выборке: 0.8149300155520995 
Глубина дерева: 9 
Количество деревьев 14


### Логистическая регрессия

In [14]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 

model.fit(train_features, train_target)

result = model.score(valid_features, valid_target)

print('Качество модели на валидационной выборке:', result)

Качество модели на валидационной выборке: 0.6842923794712286


### Вывод

Лучшее качество у модели "Случайный лес" на валидационной выборке: 0.8149300155520995 с глубиной дерева: 9 и количеством деревьев: 14

## Проверка моделей на тестовой выборке

Т к модель "Случайный лес" и "Дерево решений" получили близкие значения качества более 0.75, проверим обе модели на тестовой выборке:

In [15]:
model = RandomForestClassifier(random_state=12345, criterion='gini', n_estimators=est, max_depth=depth)
model.fit(train_features, train_target)
result = model.score(test_features, test_target)

print('Качество модели "Случайный лес" на тестовой выборке:', result)

Качество модели "Случайный лес" на тестовой выборке: 0.7916018662519441


In [16]:
model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
model.fit(train_features, train_target) 
result = model.score(test_features, test_target)
print('Качество модели "Дерево решений" на тестовой выборке:', result)

Качество модели "Дерево решений" на тестовой выборке: 0.7822706065318819


### Вывод:
Лучшая модель "Случайный лес": качество на на тестовой выборке: 0.7916018662519441, на валидационной выборке: 0.8149300155520995

## Проверка модели на адекватность

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной. 

In [17]:
adequacy_test = DummyClassifier(strategy="uniform" ,random_state=12345)
adequacy_test.fit(train_features, train_target)
result_accuracy = adequacy_test.score(test_features, test_target)
print('Качество dummy модели на тестовой выборке:', result_accuracy)

Качество dummy модели на тестовой выборке: 0.5038880248833593


### Вывод:
Сравнивая на тестовой выборке качество модели "Случайный лес" (0.7916018662519441) с dummy моделью (0.5038880248833593), приходим к выводу, что модель "Случайный лес" на самом деле достаточно качественно прогнозирует.

## Общий вывод

1. Всего в данных 3214 строки, в каждой содержится информация по каждому пользователю об используемых услугах мобильной связи «Мегалайн» за месяц. Пропуски, нарушения стиля, неверный тип данных и аномалии не обнаружены. Предобработка не требуется.
2. Данные разделили на тренировочную (60%), валидационную (20%) и тестовую (20%) выборки. 
3. Определили целевой признак - тариф, колонка 'is_ultra'.
4. Т к значений целевого признака всего два - определим тип задачи - бинарная классификация.
5. Выборки готовы, признаки и целевой признак для каждой выборки извлечены (train - выборка для обучения модели, valid - валидационная выборка и test - тестовая выборка)
6. Для решения задачи обучили три типа моделей и проверли качество прогнозирования:
* Дерево решений
* Случайный лес
* Логистическая регрессия

7. **Лучшее качество у модели "Случайный лес" на валидационной выборке: 0.8149300155520995 с глубиной дерева: 9 и количеством деревьев: 14**
8. **Худшее качество у модели "Логистическая регрессия": 0.6842923794712286**
9. **Т к модель "Случайный лес" и "Дерево решений" получили близкие значения качества более 0.75, проверили обе модели на тестовой выборке - лучшая модель "Случайный лес": качество на на тестовой выборке: 0.7916018662519441, на валидационной выборке: 0.8149300155520995**
10. **Проверили модель на вменяемость, качество модели достаточно адекватное**